In [1]:
from abc import ABCMeta, abstractmethod


class TSP():
    """
    Class to hold a TSP, sub-class will implement different improvement
    heuristics.
    """
    __metaclass__ = ABCMeta

    edges = {}  # Global cost matrix
    ratio = 10.  # Global ratio
    routes = {}  # Global routes costs

    def __init__(self, nodes, fast=False):
        """
        Initialise a TSP instance based on a scenario.

        Parameters:

            - nodes: nodes in the scenario
        """
        self.nodes = nodes
        self.fast = fast

        self.initial_path = nodes
        self.initial_cost = self.pathCost(nodes)
        # Do not save the initial path as it is not optimised
        self.heuristic_path = self.initial_path
        self.heuristic_cost = self.initial_cost

    def save(self, path, cost):
        """
        Save the heuristic cost and path.

        Parameters:

            - path: path

            - cost: cost of the path
        """
        self.heuristic_path = path
        self.heuristic_cost = cost

        self.routes[str(sorted(path))] = {"path": path, "cost": cost}

    def update(self, solution):
        """
        Update the heuristic solution with the master solution.

        Parameters:

            - solution: current master solution

        Updating the path should always be done on the initial path.

        >>> from tsp_local.test import TSPTest, matrix
        >>> TSP.setEdges(matrix)
        >>> l = list(range(len(matrix)))
        >>> t = TSPTest(l)
        >>> t.update(l[2:])
        >>> set(t.heuristic_path) == set([2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
        True
        >>> t.update(l[:2])
        >>> set(t.heuristic_path) == set([0, 1])
        True
        >>> t.update(l[2:7])
        >>> set(t.heuristic_path) == set([2, 3, 4, 5, 6])
        True
        """
        self.heuristic_path = [i for i in self.initial_path if i in solution]
        self.heuristic_cost = self.pathCost(self.heuristic_path)

    def __str__(self):
        out = "Route with {} nodes ({}):\n".format(
            len(self.heuristic_path), self.heuristic_cost)

        if self.heuristic_cost > 0:
            out += " -> ".join(map(str, self.heuristic_path))
            out += " -> {}".format(self.heuristic_path[0])
        else:
            out += "No current route."

        return out

    @staticmethod
    def dist(i, j):
        return TSP.edges[i][j]

    @staticmethod
    def pathCost(path):
        # Close the loop
        cost = TSP.dist(path[-1], path[0])

        for i in range(1, len(path)):
            cost += TSP.dist(path[i - 1], path[i])

        return cost

    @staticmethod
    def setRatio(ratio):
        TSP.ratio = ratio

    @staticmethod
    def setEdges(edges):
        TSP.edges = edges

    def optimise(self):
        """
        Check if the current route already exists before optimising.

        >>> from tsp_local.test import TSPTest, matrix
        >>> l = list(range(4))
        >>> TSP.setEdges(matrix)
        >>> t = TSPTest(l)
        >>> t.routes[str(sorted(l))] = {"path": l, "cost": 16}
        >>> t.heuristic_path = l
        >>> t.optimise()
        ([0, 1, 2, 3], 16)
        """
        route = str(sorted(self.heuristic_path))

        if route in self.routes:
            saved = TSP.routes[route]
            self.heuristic_path = saved["path"]
            self.heuristic_cost = saved["cost"]
        else:
            self._optimise()

        return self.heuristic_path, self.heuristic_cost

    @abstractmethod
    def _optimise(self):
        """
        Use an optimisation heuristic on the current TSP.
        """
        pass





In [2]:
import logging

logging.basicConfig()
Log = logging.getLogger("TSP")

In [3]:
cross = [[0, 2, 3, 2], [2, 0, 2, 3], [3, 2, 0, 2], [2, 3, 2, 0]]
start = [0, 2, 1, 3]


def swap(path, i, j):
    """
    Swap two elements in a list and reverse what was in between.

    >>> swap([1, 2, 3, 4, 5], 1, 2)
    [1, 3, 2, 4, 5]
    >>> swap([1, 2, 3, 4, 5], 0, 3)
    [4, 3, 2, 1, 5]
    >>> swap([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 0, 1)
    [1, 0, 2, 3, 4, 5, 6, 7, 8, 9]
    >>> swap([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 2, 6)
    [0, 1, 6, 5, 4, 3, 2, 7, 8, 9]
    >>> swap(start, 0, 2)
    [1, 2, 0, 3]
    """

    return path[:i] + list(reversed(path[i:j + 1])) + path[j + 1:]


class TwoOpt(TSP):
    """
    Implement the 2-opt operator for the TSP.

    Cross optimisation
    >>> TSP.setEdges(cross)
    >>> t = TwoOpt(range(4))
    >>> t.heuristic_path = start
    >>> t.heuristic_cost = 10
    >>> t._optimise()
    >>> t.heuristic_cost
    8
    """

    def _optimise(self):
        """
        U.S. test.

        >>> from tsp_local.test import matrix
        >>> TSP.setEdges(matrix)
        >>> l = list(range(len(matrix)))
        >>> t = TwoOpt(l)

        Force initial solution.
        >>> t.heuristic_path = l
        >>> t.heuristic_cost = t.pathCost(l) # 18703
        >>> t._optimise()
        >>> t.heuristic_cost < t.pathCost(l)
        True
        """
        bestChange = -1
        bestPath = self.heuristic_path
        size = len(bestPath)

        while bestChange < 0:
            saved, bestChange = self._improve(bestPath, size)

            if bestChange < 0:
                i, j = saved  # `i` is the last element in place
                bestPath = swap(bestPath, i + 1, j)

        self.save(bestPath, self.pathCost(bestPath))

    def _improve(self, bestPath, size):
        bestChange = 0
        saved = None

        for n in range(size - 3):
            for m in range(n + 2, size - 1):
                i = bestPath[n]
                j = bestPath[m]
                k = bestPath[n + 1]
                l = bestPath[m + 1]

                # Replacement arcs are:
                #  * i -> k => i -> j
                #  * j -> l => k -> l
                change = self.dist(i, j) + self.dist(k, l)
                change -= self.dist(i, k) + self.dist(j, l)

                if change < bestChange:
                    bestChange = change
                    saved = (n, m)

                    # If fast, we return the first improving move
                    if self.fast:
                        return saved, bestChange
                    # Otherwise, we explore all possible moves

        return saved, bestChange



In [16]:
import numpy as np

file = open(r"/Users/alef/Downloads/Medium.txt")
lines = file.readlines()[:-1]
points = [line.split() for line in lines]
points = [[float(x),float(y)] for _,x,y in points]

points = np.array(points)
z = np.array([complex(c[0], c[1]) for c in points])
m, n = np.meshgrid(z, z)
matrix = abs(m-n)


TSP.setEdges(matrix)
l = list(range(len(matrix)))
t = TwoOpt(l)

t.heuristic_path = l
t.heuristic_cost = t.pathCost(l)
t._optimise()


KeyboardInterrupt: 

In [15]:
t.heuristic_path, t.heuristic_cost

[0, 7, 2, 3, 9, 10, 5, 4, 11, 1, 8, 6, 12]